In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [ ]:
df = pd.read_csv("/kaggle/input/e-commerce-dataset/FashionDataset.csv")
df.head()

In [ ]:
print("Exploring the d_set  : \n")
print("Information of the dataset :\n", df.info(),"\n")
print("Shape of the dataset: \n",df.shape ,"\n")
print("Description :\n",df.describe(),"\n")
print("Data type of each column :\n",df.dtypes,"\n")
print("Head of the dataset :\n",df.head(5),"\n")
print("Tail of the dataset :\n",df.tail(5),"\n")
print("Some random sample :\n",df.sample(15),"\n")

In [ ]:
#metadata
df.columns.tolist() 

**Cleaning and preprocessing**

In [ ]:
df.isnull().sum()   

In [ ]:
df1 = df.drop_duplicates()
df1.shape

In [ ]:
df["Sizes"] = df["Sizes"].str.replace("Size:","",regex=True)
df["Sizes"] = df["Sizes"].str.split(",").astype(str)

In [ ]:
df["MRP"]=df["MRP"].astype(str).str.replace("Rs\n",  "").astype(float)
df["SellPrice"] = df["SellPrice"].astype(float)

df["Discount"] = df["Discount"].astype(str).str.replace("% off",  "").astype(float)/100
df["Category"] = df["Category"].str.replace("-Women","").astype("category")

df["Deatils"] = df["Deatils"].astype("string")
df["BrandName"] = df["BrandName"].astype(str)

In [ ]:
df = df.drop("Unnamed: 0",axis="columns")

In [ ]:
df.sample(10)

In [ ]:
df = df.dropna(subset=["MRP"])
'''After dropping Nan values from MRP the Nan values from Sellprice and Discount also removed because 
They were related'''

In [ ]:
df.sample(10)

In [ ]:
def unique_data_count(df):
    print("*************************************")
    print("Number of unique data in each column :")
    print("*************************************")
    print("Brandname :",df.BrandName.nunique(),"\n")
    print("Details: ",df.Deatils.nunique(),"\n")
    print("Sizes :",df.Sizes.nunique(),"\n")
    print("Category :",df.Category.nunique(),"\n")
unique_data_count(df)

In [ ]:
df.rename(columns={'Deatils': 'Details'}, inplace=True)

In [ ]:
df["Category"].value_counts().plot(kind="bar",figsize=(9, 5))
plt.xlabel("Category")
plt.ylabel("Count")
plt.title("Count based on category")
plt.show()

In [ ]:
from scipy.stats import skew, kurtosis
def Sellprice_analysis(df):
    print(df["SellPrice"].describe())

    #looking for Distribution of the data 
    plt.figure(figsize=(8, 2))
    sns.boxplot(x=df["SellPrice"], color="lightgreen")
    plt.title("Boxplot of SellPrice")
    plt.xlabel("SellPrice")
    plt.show()

    #Boxplot to spot outliers
    plt.figure(figsize=(10, 5))
    sns.histplot(df["SellPrice"], bins=20, kde=True, color="red")
    plt.title("Distribution of SellPrice")
    plt.xlabel("SellPrice")
    plt.ylabel("Count")
    plt.show()

    #Numerical sense of distribution
    print("**********************")
    print("Skewness and Kurtosis")
    print("**********************")
    print("Skewness:", skew(df["SellPrice"].dropna()))
    print("Kurtosis:", kurtosis(df["SellPrice"].dropna()))
    

Sellprice_analysis(df)

In [ ]:
Q1 = df["SellPrice"].quantile(0.25)
Q3 = df["SellPrice"].quantile(0.75)

IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df = df[(df["SellPrice"] >= lower_bound) & (df["SellPrice"] <= upper_bound)]


print("********************************************")
print("Skewness and kurtosis after outlier removal ")
print("********************************************")
print("Skewness:", skew(df["SellPrice"].dropna()))
print("Kurtosis:", kurtosis(df["SellPrice"].dropna()))


In [ ]:
#Brand with less than 5 products will be treated as other from now 
df["BrandName"].nunique()
brand_counts = df["BrandName"].value_counts()
rare_brands = brand_counts[brand_counts < 5].index
df["BrandName"] = df["BrandName"].apply(lambda x: "Other" if x in rare_brands else x)
df.shape

In [ ]:
# Duplicates
print("Duplicate Rows:", df.duplicated().sum())


#Negative or zero  values 
for col in df.select_dtypes(include=["float64","int64"]).columns:
    if(df[col] <= 0).any():
        print(f"Warning The data in {col} is zero or negative ")

# Categorical inconsistncies
for col in df.select_dtypes(include=['object']).columns:
    print(f"Unique values in {col}:", df[col].unique())


In [ ]:
# Classification: Check class imbalance
print("Target Distribution:")
print(df['SellPrice'].value_counts(normalize=True))

# Regression: target distribution
if df['SellPrice'].dtype in ['float64', 'int64']:
    sns.histplot(df['SellPrice'], kde=True)
    plt.title("Target Variable Distribution")
    plt.show()


In [ ]:
from scipy.stats import f_oneway, chi2_contingency

# ANOVA--> numerical features vs target (classification)
anova_results = {}
for col in df.select_dtypes(include=['float64', 'int64']).columns:
    if col != 'SellPrice':
        groups = [group[col].dropna() for name, group in df.groupby('SellPrice')]
        if len(groups) > 1:
            stat, p = f_oneway(*groups)
            anova_results[col] = p
anova_results = {k: v for k, v in anova_results.items() if v < 0.05}
print("Significant features based on ANOVA (p < 0.05):", anova_results)

# Chi-squared--> categorical features vs target
for col in df.select_dtypes(include=['object', 'category']).columns:
    if col != 'SellPrice':
        contingency = pd.crosstab(df[col], df['SellPrice'])
        chi2, p, dof, ex = chi2_contingency(contingency)
        print(f"Chi-square test for {col} vs target: p-value = {p:.4f}")


In [ ]:
df.sample(10)

In [ ]:
brand_avg_discount = df.groupby("BrandName")["Discount"].mean().sort_values(ascending=False)
top_brands = brand_avg_discount.head(40) 

plt.figure(figsize=(15, 10))
sns.barplot(x=top_brands.values, y=top_brands.index)
plt.xlabel("Average Discount %")
plt.title("Top 20 Discount-Heavy Brands")
plt.tight_layout()
plt.show()


plt.figure(figsize=(8, 5))
sns.histplot(df["Discount"], bins=50, kde=True)
plt.title("Distribution of Discount Percent")
plt.xlabel("Discount %")
plt.ylabel("Frequency")
plt.show()

In [ ]:
#Brand Average sellprice
Brand_avg_sellprice = df.groupby("BrandName")["SellPrice"].mean().sort_values(ascending=False)
Top_brands = Brand_avg_sellprice.head(30)

plt.figure(figsize=(10,6))
sns.barplot(x=Top_brands.values,y=Top_brands.index)
plt.xlabel("Top Brand Names")
plt.title("Top 30 High Price brands")
plt.show()


plt.figure(figsize=(15,8))
sns.histplot(df["SellPrice"],bins=30,kde=True)
plt.xlabel("Sellprice")
plt.title("Distribution of SellPrice")
plt.show()

In [ ]:
df.sample(10)

In [ ]:
#Relation between Category and MRP
plt.figure(figsize=(10,6))
sns.scatterplot(x = df["SellPrice"],y = df["Category"],color="black")
plt.xlabel("SellPrice")
plt.ylabel("Category")
plt.show()

In [ ]:
#Discount On CAtegory
sns.scatterplot(x = df["Discount"],y = df["Category"])

In [ ]:
#Histrogram view for numeric featutes
df.select_dtypes(include=['float64', 'int64']).hist(figsize=(15, 10), bins=30)
plt.tight_layout()
plt.suptitle("Histograms of Numeric Features", y=1.02)
plt.show()

#Correlation matrix 
plt.figure(figsize=(12, 8))
corr = df.select_dtypes(include=['float64', 'int64']).corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix Heatmap")
plt.show()


In [ ]:
contingency_table = pd.crosstab(df['BrandName'], df['Category'])

#Display the contingency table head
print("Contingency Table (BrandName vs. Category):\n", contingency_table.head())


#top 40 brands for readability
plt.figure(figsize=(15, 10))
sns.heatmap(contingency_table.head(50), annot=True, cmap='YlGnBu', fmt='d')
plt.title('Heatmap of BrandName vs. Category (Top 20 Brands)')
plt.xlabel('Category')
plt.ylabel('BrandName')
plt.show()

#Perform Chi-squared test
chi2, p, dof, ex = chi2_contingency(contingency_table)

print(f"\nChi-squared test for BrandName vs. Category:")
print(f"Chi2 Statistic: {chi2:.2f}")
print(f"P-value: {p:.4f}")
print(f"Degrees of Freedom: {dof}")

#theory for   the p-value
alpha = 0.05
if p < alpha:
    print("\nSince the p-value is less than the significance level (0.05), we reject the null hypothesis.")
    print("There is a statistically significant relationship between BrandName and Category.")
else:
    print("\nSince the p-value is greater than the significance level (0.05), we fail to reject the null hypothesis.")
    print("There is no statistically significant relationship between BrandName and Category.")

In [ ]:
# Save the cleaned dataset to a CSV file
df.to_csv("/kaggle/working/Cleaned_FashionDataset.csv", index=False)


##  EDA Summary – Fashion Dataset (Dynamic Pricing Project)

This notebook is part of my project to build a dynamic pricing engine. I performed Exploratory Data Analysis (EDA) on a fashion e-commerce dataset (`FashionDataset.csv`) to understand the structure, clean the data, and get it ready for modeling.

*********

###  Libraries Used:
-->> `pandas` – for handling and cleaning data
-->> `numpy` – for numeric operations
-->> `matplotlib` & `seaborn` – for creating visualizations
-->> `scipy.stats` – for statistical tests like ANOVA and Chi-squared

************

###  Main Steps:
-->> Loaded the dataset and checked structure using `.head()`, `.info()`, `.describe()`
-->> Removed duplicates and unnecessary columns (`Unnamed: 0`)
-->> Cleaned and converted(Some data types) columns: `Sizes`, `MRP`, `SellPrice`, `Discount`,             'Category`, and `BrandName`.
-->> Removed rows with missing `MRP`, and handled outliers in `SellPrice` using IQR
-->> Grouped rare `BrandName` values into `'Other'` for better analysis

**********

###  Problems Faced:
-->> **Inconsistent data types** (like strings with text in numeric columns)
-->> **Outliers** in `SellPrice` were affecting distribution
-->> **Too many unique brands**, so I grouped the less frequent ones
-->> **Reset environment** required re-running preprocessing during work

*********

###  Key Findings:
-->> `MRP` and `Discount` are strong predictors of `SellPrice`
-->> Strong link between `BrandName` and `Category` (confirmed using Chi-squared test)
-->> `Westernwear` is the most dominant category
-->> Brands often specialize in specific categories (e.g., 'ajmal' in Fragrance)

**********

###  Usable for Modeling:
-->> Cleaned `MRP`, `SellPrice`, and `Discount` (numeric and usable)
-->> Simplified `BrandName` and `Category` (less cardinality, more meaningful)
-->> Dataset is ready for feature engineering and training ML models

*****

###  What I’ll Do Next with This Dataset:
Now that EDA is done, the next steps in my project are:

- **Feature Engineering** : I’ll transform the cleaned data into features that machine learning models can understand. For example:
  - Extract useful keywords or attributes from the `Details` column using text processing
  - Convert categorical columns like `BrandName` and `Category` into numeric features (one-hot encoding or label encoding).
  - Handle the `Sizes` column more deeply, possibly by creating size indicators or most common size tags.
  
- **Model Training**: Use the features to train a regression model that predicts `SellPrice` based on inputs like `MRP`, `Discount`, `Brand`, `Category`, and possibly some text-derived features.


This EDA has helped shape a solid foundation for building that pricing engine.

*****

###  Saved Cleaned Dataset
The cleaned dataset was saved as:

```python
df.to_csv("/kaggle/working/Cleaned_FashionDataset.csv", index=False)
